In [ ]:
## On ‘Wuzzuf_Job_Postings_Sample.csv’, the following has been done:
## I used OpenRefine to:
## Remove all the rows with missing values in job_description, job_requirements, 
##         payment_period, currency (21850 rows to 19112 rows)
## Clustered all the similar cities in city, but kept certain data that were dates as is - 
##        “10th of Ramadan”, “6th October”, etc., as it is uninformative on which 
##        city it refers to
## the input data here is the cleaned dataset
## ----------------------------------------------------------------------------------------------
## I have also done a few pre-modeling Vizs with a couple of good insights which you can
## check out on: 
## https://public.tableau.com/profile/pooja.vijaykumar#!/vizhome/WuzzufJobPostingsDataset/Story1
## ----------------------------------------------------------------------------------------------
## A Classifier can be created that classifies each application based on job_category1 so that 
## when a person enters their qualifications, max/min salary expectations, their skills, etc., 
## their application falls into the respective category such as Engineering, IT, Marketing, 
## HR, Banking, Retail, Fashion, Journalism etc., 
## Features: salary min/max, payment_period, currency, num_vacancies, career_level
## Target: job_category1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

In [ ]:
print(os.listdir('../input'))

In [ ]:
data1 = pd.read_csv('../input/cleaned-wuzzuf-job-posts-sample/Cleaned_Wuzzuf_Job_Posts_Sample-csv.csv')

In [ ]:
data2 = pd.read_csv('../input/wuzzuf-job-posts/Wuzzuf_Applications_Sample.csv')

In [ ]:
data1.head()

In [ ]:
data1.info()

In [ ]:
## check for missing values
data1.isnull().sum()

In [ ]:
## remove record with the single missing value
data1 = data1.dropna(axis=0)

In [ ]:
data2.head()

In [ ]:
data2.info()

### Saving data into another variable so as not to modify the original dataset: (Rather than dropping features from the original dataset, it's sometimes better to take all the necessary features and save it into another variable)

In [ ]:
data11 = data1[['job_category1','job_industry1','salary_minimum','salary_maximum','num_vacancies','career_level','payment_period','currency']]

In [ ]:
data11.head(3)

### One-hot encoding of all the object type columns:

In [ ]:
data11 = data11.join(pd.get_dummies(data11['career_level'],prefix='CareerLevel'))

In [ ]:
data11 = data11.join(pd.get_dummies(data11['payment_period'],prefix='paymentperiod'))

In [ ]:
data11 = data11.join(pd.get_dummies(data11['currency'],prefix='currency'))

### Removing certain columns to avoid Multicollinearity:

In [ ]:
data11 = data11.drop(['job_industry1','career_level','payment_period','currency'],axis=1)

In [ ]:
data11.head()

In [ ]:
x = data11.drop('job_category1',axis=1)

In [ ]:
y = data11['job_category1']

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=123)

### Decision Tree Classifier:

In [ ]:
DTmodel = DecisionTreeClassifier(max_depth=4).fit(xtrain,ytrain)

In [ ]:
DTpred = DTmodel.predict(xtest)

In [ ]:
DTpred[:5,]

In [ ]:
ytest[:5,]

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(ytest,DTpred)

In [ ]:
## Run this if you want to observe the Decision Tree diagram
## useful to observe the feature split, gini index, etc

#from sklearn.tree import export_graphviz
#import graphviz

#dot_data = export_graphviz(DTmodel, filled=True, rounded=True, feature_names=xtrain.columns, out_file=None)
#graphviz.Source(dot_data)

### Decision Tree Classifier with GridSearchCV:

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param = [{"max_depth":[5,6,7,8,9,10,11,12,13,14, None], "max_features":[7,10,11,12,13,14,15,16]}]

In [ ]:
gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=321),param_grid=param,cv=2)

In [ ]:
gs.fit(xtrain,ytrain)

In [ ]:
gs.best_params_

In [ ]:
pred = gs.predict(xtest)

In [ ]:
accuracy_score(ytest,pred)

### Accuracy score has increased from 0.34 to 0.36 using GridSearchCV

### Random Forest Classifier: (because it's always better to compare algorithms)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFmodel = RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [ ]:
RFmodel.fit(xtrain, ytrain)

In [ ]:
RFpred=RFmodel.predict(xtest)

In [ ]:
accuracy_score(ytest,RFpred)

### Random Forest shows higher accuracy score when compared to Decision Tree

### I've tried what I could with what I've learnt so far in this field. Please do let me know if there's anything else that can be done. The fun is that there are thousands of opportunities that come with a dataset like this! Different models can be built for different intentions so go crazy!
    